In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.expand_frame_repr', None)
pd.set_option('max_colwidth', -1)

In [2]:
#!pip install pulp

In [3]:
#df_1 = pd.read_csv('ind_sl_t20i_3rd_2022_02_27.txt')
#df_1.to_csv('ind_sl_t20i_3rd_2022_02_27.csv', index = None)

# 1st T20i IND vs SL

In [4]:
df = pd.read_csv('ind_sl_t20i_1st_2022_02_24.csv')
df.head(2)

,SeriesId,MatchId,Player,PlayerId,Team,TeamId,Role,Sel,Credits,Series Points,PlayingXI,Points,DT
0,3692,38632,I Kishan,10276,IND,2,WK,73.23,8.5,0,1,141,1
1,3692,38632,S Iyer,9428,IND,2,BAT,71.23,8.5,0,1,84,1


In [5]:
df = df.join(pd.get_dummies(df[['Role','Team']]))
df.head(2)

,SeriesId,MatchId,Player,PlayerId,Team,TeamId,Role,Sel,Credits,Series Points,PlayingXI,Points,DT,Role_AR,Role_BAT,Role_BOWL,Role_WK,Team_IND,Team_SL
0,3692,38632,I Kishan,10276,IND,2,WK,73.23,8.5,0,1,141,1,0,0,0,1,1,0
1,3692,38632,S Iyer,9428,IND,2,BAT,71.23,8.5,0,1,84,1,0,1,0,0,1,0


In [6]:
import pulp

pickup_status = pulp.LpVariable.dicts('pickup_status', ((obj) for obj in df.index), cat = 'Binary')

mo = pulp.LpProblem('Points Maximisation Problem', pulp.LpMaximize)
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj),'Points'] for obj in df.index])

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Credits']] for obj in df.index) <= 100

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'PlayingXI']] for obj in df.index) == 11

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Team_IND']] for obj in df.index) >= 4
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Team_IND']] for obj in df.index) <= 7

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Team_SL']] for obj in df.index) >= 4
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Team_SL']] for obj in df.index) <= 7

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_WK']] for obj in df.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_WK']] for obj in df.index) <= 4

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_BAT']] for obj in df.index) >= 3
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_BAT']] for obj in df.index) <= 6

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_AR']] for obj in df.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_AR']] for obj in df.index) <= 4

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_BOWL']] for obj in df.index) >= 3
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_BOWL']] for obj in df.index) <= 6


In [7]:
mo.solve()

1

In [8]:
df['pickup_status'] = 0

for obj in df.index:
    if pickup_status[obj].varValue:
        df['pickup_status'][obj] = 1

In [9]:
t20i_1_df = df[df['pickup_status'] == 1].sort_values(['Role_BAT', 'Role_WK', 'Role_AR', 'Role_BOWL'], ascending = False)
t20i_1_df.reset_index(drop=True)

,SeriesId,MatchId,Player,PlayerId,Team,TeamId,Role,Sel,Credits,Series Points,PlayingXI,Points,DT,Role_AR,Role_BAT,Role_BOWL,Role_WK,Team_IND,Team_SL,pickup_status
0,3692,38632,S Iyer,9428,IND,2,BAT,71.23,8.5,0,1,84,1,0,1,0,0,1,0,1
1,3692,38632,C Asalanka,10934,SL,5,BAT,64.60,8.5,0,1,70,1,0,1,0,0,0,1,1
2,3692,38632,R Sharma,576,IND,2,BAT,87.30,10.5,0,1,66,1,0,1,0,0,1,0,1
3,3692,38632,I Kishan,10276,IND,2,WK,73.23,8.5,0,1,141,1,0,0,0,1,1,0,1
4,3692,38632,V Iyer,10917,IND,2,AR,53.22,8.5,0,1,50,1,1,0,0,0,1,0,1
5,3692,38632,R Jadeja,587,IND,2,AR,74.73,9.0,0,1,34,1,1,0,0,0,1,0,1
6,3692,38632,D Shanaka,8422,SL,5,AR,50.75,9.0,0,1,32,1,1,0,0,0,0,1,1
7,3692,38632,B Kumar,1726,IND,2,BOWL,22.17,8.5,0,1,76,1,0,0,1,0,1,0,1
8,3692,38632,Y Chahal,7910,IND,2,BOWL,40.62,8.5,0,1,35,1,0,0,1,0,1,0,1
9,3692,38632,L Kumara,10928,SL,5,BOWL,34.56,8.5,0,1,35,1,0,0,1,0,0,1,1


In [10]:
t20i_1_df.columns, sum(t20i_1_df.Credits)

(Index(['SeriesId', 'MatchId', 'Player', 'PlayerId', 'Team', 'TeamId', 'Role',
        'Sel', 'Credits', 'Series Points', 'PlayingXI', 'Points', 'DT',
        'Role_AR', 'Role_BAT', 'Role_BOWL', 'Role_WK', 'Team_IND', 'Team_SL',
        'pickup_status'],
       dtype='object'),
 97.0)

In [28]:
match1_avg_points = t20i_1_df.Points.mean()
match1_sum_points = sum(t20i_1_df.Points)

print('Average Points in the 1st T20 match between IND vs SL: ', match1_avg_points)
print('Sum of total points in the 1st T20 match between IND vs SL: ', match1_sum_points)

Average Points in the 1st T20 match between IND vs SL:  59.36363636363637
Sum of total points in the 1st T20 match between IND vs SL:  653


# 2nd T20i IND vs SL

In [29]:
df1 = pd.read_csv('ind_sl_t20i_2nd_2022_02_26.csv')
df1.head(2)

,SeriesId,MatchId,Player,PlayerId,Team,TeamId,Role,Sel,Credits,Series Points,Series DT,PlayingXI,Points,DT
0,3692,38637,S Iyer,9428,IND,2,BAT,87.64,8.5,84,1,1,112,1
1,3692,38637,P Nissanka,13682,SL,5,BAT,66.77,9.5,2,0,1,100,1


In [31]:
df1 = df1.join(pd.get_dummies(df1[['Role','Team']]))
df1.head(2)

,SeriesId,MatchId,Player,PlayerId,Team,TeamId,Role,Sel,Credits,Series Points,Series DT,PlayingXI,Points,DT,Role_AR,Role_BAT,Role_BOWL,Role_WK,Team_IND,Team_SL
0,3692,38637,S Iyer,9428,IND,2,BAT,87.64,8.5,84,1,1,112,1,0,1,0,0,1,0
1,3692,38637,P Nissanka,13682,SL,5,BAT,66.77,9.5,2,0,1,100,1,0,1,0,0,0,1


In [45]:
pickup_status = pulp.LpVariable.dicts('pickup_status', ((obj) for obj in df1.index), cat = 'Binary')

mo = pulp.LpProblem('Points Maximization Problem', pulp.LpMaximize)
mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), 'Points'] for obj in df1.index])

mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), 'Credits']] for obj in df1.index) <= 100

mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), 'PlayingXI']] for obj in df1.index) == 11

mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), 'Team_IND']] for obj in df1.index) >= 4
mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), 'Team_IND']] for obj in df1.index) <= 7

mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), 'Team_SL']] for obj in df1.index) >= 4
mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), 'Team_SL']] for obj in df1.index) <= 7

mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), 'Role_WK']] for obj in df1.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), 'Role_WK']] for obj in df1.index) <= 4

mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), 'Role_BAT']] for obj in df1.index) >= 3
mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), 'Role_BAT']] for obj in df1.index) <= 6

mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), 'Role_AR']] for obj in df1.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), 'Role_AR']] for obj in df1.index) >= 4

mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), 'Role_BOWL']] for obj in df1.index) >= 3
mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), 'Role_BOWL']] for obj in df1.index) <= 6


In [46]:
mo.solve()

1

In [49]:
df1['pickup_status'] = 0

for obj in df1.index:
    if pickup_status[obj].varValue:
        df1['pickup_status'][obj] = 1

In [50]:
t20i_2_df = df1[df1['pickup_status'] == 1].sort_values(['Role_BAT', 'Role_WK', 'Role_AR', 'Role_BOWL'], ascending = False)
t20i_2_df.reset_index(drop=True)

,SeriesId,MatchId,Player,PlayerId,Team,TeamId,Role,Sel,Credits,Series Points,Series DT,PlayingXI,Points,DT,Role_AR,Role_BAT,Role_BOWL,Role_WK,Team_IND,Team_SL,pickup_status
0,3692,38637,S Iyer,9428,IND,2,BAT,87.64,8.5,84,1,1,112,1,0,1,0,0,1,0,1
1,3692,38637,P Nissanka,13682,SL,5,BAT,66.77,9.5,2,0,1,100,1,0,1,0,0,0,1,1
2,3692,38637,D Gunathilaka,8387,SL,5,BAT,4.45,8.5,0,0,1,56,1,0,1,0,0,0,1,1
3,3692,38637,S Samson,8271,IND,2,WK,25.94,8.0,0,0,1,59,1,0,0,0,1,1,0,1
4,3692,38637,R Jadeja,587,IND,2,AR,76.24,9.0,34,1,1,93,1,1,0,0,0,1,0,1
5,3692,38637,D Shanaka,8422,SL,5,AR,68.26,9.0,32,1,1,77,1,1,0,0,0,0,1,1
6,3692,38637,V Iyer,10917,IND,2,AR,65.32,8.5,50,1,1,12,0,1,0,0,0,1,0,1
7,3692,38637,D Hooda,9427,IND,2,AR,7.79,8.0,4,0,1,4,0,1,0,0,0,1,0,1
8,3692,38637,L Kumara,10928,SL,5,BOWL,46.78,8.5,35,1,1,52,1,0,0,1,0,0,1,1
9,3692,38637,Y Chahal,7910,IND,2,BOWL,54.21,8.5,35,1,1,39,1,0,0,1,0,1,0,1


In [53]:
t20i_2_df.columns, sum(t20i_2_df.Credits)

(Index(['SeriesId', 'MatchId', 'Player', 'PlayerId', 'Team', 'TeamId', 'Role',
        'Sel', 'Credits', 'Series Points', 'Series DT', 'PlayingXI', 'Points',
        'DT', 'Role_AR', 'Role_BAT', 'Role_BOWL', 'Role_WK', 'Team_IND',
        'Team_SL', 'pickup_status'],
       dtype='object'),
 94.5)

In [52]:
match2_average_points = t20i_2_df.Points.mean()
match2_sum_points = t20i_2_df.Points.sum()

print('Average points in the 2nd match between IND vs SL: ', match2_average_points)
print('Sum of points in the 2nd match between IND vs SL: ', match2_sum_points)

Average points in the 2nd match between IND vs SL:  58.27272727272727
Sum of points in the 2nd match between IND vs SL:  641


In [62]:
df2 = pd.read_csv('ind_sl_t20i_3rd_2022_02_27.csv')
df2.head(2)

,SeriesId,MatchId,Player,PlayerId,Team,TeamId,Role,Sel,Credits,Series Points,Series DT,PlayingXI,Points
0,3692,38642,S Iyer,9428,IND,2,BAT,88.8,8.5,196.0,2,1,100
1,3692,38642,R Jadeja,587,IND,2,AR,87.6,9.0,127.0,2,1,31


In [63]:
df2 = df2.join(pd.get_dummies(df2[['Role','Team']]))
df2.head(2)

,SeriesId,MatchId,Player,PlayerId,Team,TeamId,Role,Sel,Credits,Series Points,Series DT,PlayingXI,Points,Role_AR,Role_BAT,Role_BOWL,Role_WK,Team_IND,Team_SL
0,3692,38642,S Iyer,9428,IND,2,BAT,88.8,8.5,196.0,2,1,100,0,1,0,0,1,0
1,3692,38642,R Jadeja,587,IND,2,AR,87.6,9.0,127.0,2,1,31,1,0,0,0,1,0


In [69]:
pickup_status = pulp.LpVariable.dicts('pickup_status', ((obj) for obj in df2.index), cat = 'Binary')

mo = pulp.LpProblem('Points Maximization Problem', pulp.LpMaximize)
mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), 'Points'] for obj in df2.index])

mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), 'Credits']] for obj in df2.index) <= 100

mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), 'PlayingXI']] for obj in df2.index) == 11

mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), 'Team_IND']] for obj in df2.index) >= 4
mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), 'Team_IND']] for obj in df2.index) <= 7

mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), 'Team_SL']] for obj in df2.index) >= 4
mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), 'Team_SL']] for obj in df2.index) <= 7

mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj),'Role_WK']] for obj in df2.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), 'Role_WK']] for obj in df2.index) <= 4

mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), 'Role_BAT']] for obj in df2.index) >= 3
mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), 'Role_BAT']] for obj in df2.index) <= 6

mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), 'Role_AR']] for obj in df2.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), 'Role_AR']] for obj in df2.index) <= 4

mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), 'Role_BOWL']] for obj in df2.index) >= 3
mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), 'Role_BOWL']] for obj in df2.index) <= 6


In [70]:
mo.solve()

1

In [71]:
df2['pickup_status'] = 0

for obj in df2.index:
    if pickup_status[obj].varValue:
        df2['pickup_status'][obj] = 1

In [72]:
t20i_3_df = df2[df2['pickup_status'] == 1].sort_values(['Role_BAT', 'Role_WK', 'Role_AR', 'Role_BOWL'], ascending = False).reset_index(drop=True)

In [73]:
t20i_3_df

,SeriesId,MatchId,Player,PlayerId,Team,TeamId,Role,Sel,Credits,Series Points,Series DT,PlayingXI,Points,Role_AR,Role_BAT,Role_BOWL,Role_WK,Team_IND,Team_SL,pickup_status
0,3692,38642,S Iyer,9428,IND,2,BAT,88.80,8.5,196.0,2,1,100,0,1,0,0,1,0,1
1,3692,38642,C Asalanka,10934,SL,5,BAT,57.96,8.5,76.0,1,1,9,0,1,0,0,0,1,1
2,3692,38642,K Mishara,13886,SL,5,BAT,2.10,8.0,24.0,0,0,0,0,1,0,0,0,1,1
3,3692,38642,S Samson,8271,IND,2,WK,85.50,8.0,59.0,1,1,35,0,0,0,1,1,0,1
4,3692,38642,D Chandimal,6245,SL,5,WK,25.80,8.5,30.0,0,1,36,0,0,0,1,0,1,1
5,3692,38642,D Shanaka,8422,SL,5,AR,82.30,9.0,109.0,2,1,105,1,0,0,0,0,1,1
6,3692,38642,C Karunaratne,7952,SL,5,AR,14.50,8.5,31.0,0,1,47,1,0,0,0,0,1,1
7,3692,38642,D Hooda,9427,IND,2,AR,6.80,8.0,8.0,0,1,30,1,0,0,0,1,0,1
8,3692,38642,L Kumara,10928,SL,5,BOWL,60.44,8.5,87.0,2,1,60,0,0,1,0,0,1,1
9,3692,38642,R Bishnoi,14659,IND,2,BOWL,48.93,8.0,0.0,0,1,37,0,0,1,0,1,0,1


In [74]:
sum(t20i_3_df.Credits)

100.0

In [76]:
match3_average_points = t20i_3_df.Points.mean()
match3_sum_points = t20i_3_df.Points.sum()

print('Average Points in the 3rd T20 between IND vs SL: ', match3_average_points)
print('Sum Points in the 3rd T20 between IND vs SL: ', match3_sum_points)

Average Points in the 3rd T20 between IND vs SL:  47.5
Sum Points in the 3rd T20 between IND vs SL:  570


## Overall average points in the series between IND vs SL

In [79]:
overall_avg = (match1_avg_points + match2_average_points + match3_average_points)/3

print('Overall average: ', overall_avg)

Overall average:  55.04545454545454
